# Data Preprocessing & Feature Engineering

## Objective
Clean the dataset and engineer features for machine learning model training.

## Steps
1. Load raw data
2. Handle missing and invalid values
3. Convert data types
4. Engineer new features
5. Encode categorical variables
6. Normalize/scale numerical features
7. Save processed data

In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('../config.json', 'r') as f:
    config = json.load(f)

raw_data_path = Path(config['paths']['raw_data'])
processed_data_path = Path(config['paths']['processed_data'])

## 1. Load Raw Data

In [3]:
df = pd.read_csv(raw_data_path / 'customer_behavior.csv')
print(f"Original shape: {df.shape}")
df.head()

Original shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 2. Fix Data Types

In [4]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print(f"TotalCharges converted to numeric")
print(f"Rows with NaN TotalCharges: {df['TotalCharges'].isna().sum()}")

TotalCharges converted to numeric
Rows with NaN TotalCharges: 11


## 3. Handle Missing Values

In [5]:
missing_charges = df['TotalCharges'].isna()
print(f"\nCustomers with missing TotalCharges: {missing_charges.sum()}")

if missing_charges.sum() > 0:
    print("\nAnalyzing customers with missing charges...")
    print(df[missing_charges][['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']].head())
    
    df.loc[missing_charges, 'TotalCharges'] = df.loc[missing_charges, 'MonthlyCharges'] * df.loc[missing_charges, 'tenure']
    print(f"\nFilled missing TotalCharges with MonthlyCharges * tenure")

print(f"\nRemaining missing values: {df['TotalCharges'].isna().sum()}")


Customers with missing TotalCharges: 11

Analyzing customers with missing charges...
      customerID  tenure  MonthlyCharges  TotalCharges
488   4472-LVYGI       0           52.55           NaN
753   3115-CZMZD       0           20.25           NaN
936   5709-LVOEQ       0           80.85           NaN
1082  4367-NUYAO       0           25.75           NaN
1340  1371-DWPAZ       0           56.05           NaN

Filled missing TotalCharges with MonthlyCharges * tenure

Remaining missing values: 0


## 4. Feature Engineering

In [6]:
df['tenure_months'] = df['tenure']
df['tenure_years'] = df['tenure'] / 12

df['avg_monthly_spend'] = df['TotalCharges'] / (df['tenure'] + 1)

df['charge_per_tenure'] = df['MonthlyCharges'] / (df['tenure'] + 1)

service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

def count_services(row):
    count = 0
    for col in service_cols:
        if row[col] not in ['No', 'No internet service', 'No phone service']:
            count += 1
    return count

df['total_services'] = df.apply(count_services, axis=1)

df['has_streaming'] = ((df['StreamingTV'] == 'Yes') | (df['StreamingMovies'] == 'Yes')).astype(int)
df['has_security'] = ((df['OnlineSecurity'] == 'Yes') | (df['DeviceProtection'] == 'Yes')).astype(int)
df['has_support'] = (df['TechSupport'] == 'Yes').astype(int)

df['is_long_term'] = (df['tenure'] > 24).astype(int)
df['is_senior'] = df['SeniorCitizen']
df['has_partner_or_dependent'] = ((df['Partner'] == 'Yes') | (df['Dependents'] == 'Yes')).astype(int)

df['auto_payment'] = df['PaymentMethod'].str.contains('automatic', case=False, na=False).astype(int)

print(f"\nEngineered features:")
print(f"- Tenure-based: tenure_years")
print(f"- Financial: avg_monthly_spend, charge_per_tenure")
print(f"- Service: total_services, has_streaming, has_security, has_support")
print(f"- Customer: is_long_term, has_partner_or_dependent, auto_payment")


Engineered features:
- Tenure-based: tenure_years
- Financial: avg_monthly_spend, charge_per_tenure
- Service: total_services, has_streaming, has_security, has_support
- Customer: is_long_term, has_partner_or_dependent, auto_payment


## 5. Encode Categorical Variables

In [7]:
df['gender_encoded'] = (df['gender'] == 'Male').astype(int)
df['partner_encoded'] = (df['Partner'] == 'Yes').astype(int)
df['dependents_encoded'] = (df['Dependents'] == 'Yes').astype(int)
df['phone_service_encoded'] = (df['PhoneService'] == 'Yes').astype(int)
df['paperless_billing_encoded'] = (df['PaperlessBilling'] == 'Yes').astype(int)
df['churn_encoded'] = (df['Churn'] == 'Yes').astype(int)

contract_mapping = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
df['contract_encoded'] = df['Contract'].map(contract_mapping)

internet_mapping = {'No': 0, 'DSL': 1, 'Fiber optic': 2}
df['internet_service_encoded'] = df['InternetService'].map(internet_mapping)

payment_dummies = pd.get_dummies(df['PaymentMethod'], prefix='payment')
df = pd.concat([df, payment_dummies], axis=1)

print("\nCategorical encoding complete")
print(f"New encoded columns: {list(payment_dummies.columns)}")


Categorical encoding complete
New encoded columns: ['payment_Bank transfer (automatic)', 'payment_Credit card (automatic)', 'payment_Electronic check', 'payment_Mailed check']


## 6. Select Final Features

In [8]:
feature_columns = [
    'customerID',
    'gender_encoded',
    'SeniorCitizen',
    'partner_encoded',
    'dependents_encoded',
    'tenure',
    'tenure_years',
    'phone_service_encoded',
    'contract_encoded',
    'internet_service_encoded',
    'paperless_billing_encoded',
    'MonthlyCharges',
    'TotalCharges',
    'avg_monthly_spend',
    'charge_per_tenure',
    'total_services',
    'has_streaming',
    'has_security',
    'has_support',
    'is_long_term',
    'has_partner_or_dependent',
    'auto_payment',
    'churn_encoded'
]

payment_cols = [col for col in df.columns if col.startswith('payment_')]
feature_columns.extend(payment_cols)

df_processed = df[feature_columns].copy()

print(f"\nProcessed dataset shape: {df_processed.shape}")
print(f"Features: {len(feature_columns) - 2}")  
print(f"\nFeature list:")
for i, col in enumerate(feature_columns[1:-1], 1):
    print(f"{i:2d}. {col}")


Processed dataset shape: (7043, 27)
Features: 25

Feature list:
 1. gender_encoded
 2. SeniorCitizen
 3. partner_encoded
 4. dependents_encoded
 5. tenure
 6. tenure_years
 7. phone_service_encoded
 8. contract_encoded
 9. internet_service_encoded
10. paperless_billing_encoded
11. MonthlyCharges
12. TotalCharges
13. avg_monthly_spend
14. charge_per_tenure
15. total_services
16. has_streaming
17. has_security
18. has_support
19. is_long_term
20. has_partner_or_dependent
21. auto_payment
22. churn_encoded
23. payment_Bank transfer (automatic)
24. payment_Credit card (automatic)
25. payment_Electronic check


## 7. Save Processed Data

In [9]:
output_file = processed_data_path / 'customer_churn_processed.csv'
df_processed.to_csv(output_file, index=False)
print(f"\n✓ Processed data saved to: {output_file}")
print(f"  Shape: {df_processed.shape}")
print(f"  Size: {output_file.stat().st_size / 1024:.2f} KB")


✓ Processed data saved to: C:\Users\Khalid Abdelaty\depii r3\milestone_1\data\processed\customer_churn_processed.csv
  Shape: (7043, 27)
  Size: 874.09 KB


## 8. Feature Statistics

In [10]:
numeric_features = df_processed.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('customerID') if 'customerID' in numeric_features else None
numeric_features.remove('churn_encoded') if 'churn_encoded' in numeric_features else None

print("\nFeature Statistics:")
print("="*80)
print(df_processed[numeric_features].describe())


Feature Statistics:
       gender_encoded  SeniorCitizen  partner_encoded  dependents_encoded  \
count     7043.000000    7043.000000      7043.000000         7043.000000   
mean         0.504756       0.162147         0.483033            0.299588   
std          0.500013       0.368612         0.499748            0.458110   
min          0.000000       0.000000         0.000000            0.000000   
25%          0.000000       0.000000         0.000000            0.000000   
50%          1.000000       0.000000         0.000000            0.000000   
75%          1.000000       0.000000         1.000000            1.000000   
max          1.000000       1.000000         1.000000            1.000000   

            tenure  tenure_years  phone_service_encoded  contract_encoded  \
count  7043.000000   7043.000000            7043.000000       7043.000000   
mean     32.371149      2.697596               0.903166          0.690473   
std      24.559481      2.046623               0.29575

## 9. Data Quality Check

In [11]:
print("\nData Quality Report:")
print("="*60)
print(f"Total records: {len(df_processed)}")
print(f"Total features: {len(df_processed.columns) - 2}")
print(f"Missing values: {df_processed.isnull().sum().sum()}")
print(f"Duplicate records: {df_processed.duplicated(subset=['customerID']).sum()}")
print(f"Churn rate: {100 * df_processed['churn_encoded'].mean():.2f}%")
print("="*60)


Data Quality Report:
Total records: 7043
Total features: 25
Missing values: 0
Duplicate records: 0
Churn rate: 26.54%


## 10. Summary

In [12]:
print("\n" + "="*80)
print("PREPROCESSING SUMMARY")
print("="*80)
print(f"\n✓ Data cleaning completed")
print(f"  - Fixed TotalCharges data type")
print(f"  - Handled missing values")
print(f"\n✓ Feature engineering completed")
print(f"  - Created {len(feature_columns) - len(['customerID', 'churn_encoded'])} features")
print(f"  - Encoded categorical variables")
print(f"  - Generated derived features")
print(f"\n✓ Data saved")
print(f"  - Location: {output_file}")
print(f"  - Format: CSV")
print(f"\nNext Steps:")
print("1. Create MySQL database schema")
print("2. Load processed data into database")
print("3. Create SQL queries for feature extraction")
print("4. Validate data integrity")
print("="*80)


PREPROCESSING SUMMARY

✓ Data cleaning completed
  - Fixed TotalCharges data type
  - Handled missing values

✓ Feature engineering completed
  - Created 25 features
  - Encoded categorical variables
  - Generated derived features

✓ Data saved
  - Location: C:\Users\Khalid Abdelaty\depii r3\milestone_1\data\processed\customer_churn_processed.csv
  - Format: CSV

Next Steps:
1. Create MySQL database schema
2. Load processed data into database
3. Create SQL queries for feature extraction
4. Validate data integrity
